In [2]:
import pandas as pd

df = pd.read_csv ('tampa.csv')
print(df)

import matplotlib.pyplot as plt
import numpy as np

        Loan_Identifier  PayYear  PayMonth  OriginationYear  OriginationMonth  \
0          100004792573     2008         3             2008                 1   
1          100004792573     2012        12             2008                 1   
2          100011325973     2003         8             2003                 6   
3          100011325973     2004         8             2003                 6   
4          100011402460     2003         5             2003                 4   
...                 ...      ...       ...              ...               ...   
398221     999986301575     2004         7             2001                10   
398222     999993773646     2002         8             2002                 6   
398223     999993773646     2004         2             2002                 6   
398224     999996887539     2005        12             2005                10   
398225     999996887539     2009         3             2005                10   

               Seller_Name 

In [3]:
read_dis_df = pd.read_csv ('disasters.csv')

# filter to match what is in the city
counties = ['Hillsborough County', 'Pinellas County']
filtered_df = read_dis_df[read_dis_df['County'].isin(counties)]

# get all unique hurricane names
hurricane_names = filtered_df['Declaration Title'].str.extract(r'(HURRICANE\s+\w+)', expand=False).dropna()
hurricane_names = hurricane_names.unique()

disaster_df = pd.DataFrame({'hurricanes': hurricane_names})

# Create a dictionary mapping Hurricane names to Declaration Dates
hurricane_date_mapping = {}

for hurricane_name in disaster_df['hurricanes']:
    # Find matching Declaration Titles (considering variations)
    matching_titles = filtered_df[filtered_df['Declaration Title'].str.contains(hurricane_name, case=False)]['Declaration Title']
    if not matching_titles.empty:
        # Use the first matching Declaration Date
        hurricane_date_mapping[hurricane_name] = filtered_df.loc[filtered_df['Declaration Title'] == matching_titles.iloc[0], 'Declaration Date'].values[0]

# add in declaration date to new df
disaster_df['date'] = pd.to_datetime(disaster_df['hurricanes'].map(hurricane_date_mapping))

# add 6 months prior and after
disaster_df['-6 months'] = disaster_df['date'] - pd.DateOffset(months=6)
disaster_df['+6 months'] = disaster_df['date'] + pd.DateOffset(months=6)

# convert tampa data to date
df['t_date'] = pd.to_datetime(df[['PayYear', 'PayMonth']].assign(day=1))
df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))

print(df)

# check if in between 2 dates
#for hurricane_name in disaster_df['hurricanes']:










print(disaster_df)

ValueError: to assemble mappings requires at least that [year, month, day] be specified: [month,year] is missing